In [2]:
import os
import numpy as np
import pandas as pd
import re
import math
import pdb
import datetime
import time

# Loading StudentLife dataset

In [3]:
path = './dataset/dataset' # dir to datatset

os.listdir(path)

['app_usage',
 'calendar',
 'call_log',
 'dinning',
 'education',
 'EMA',
 'rawaccfeat',
 'sensing',
 'sms',
 'survey']

## Loading EMA data

In [4]:
''' 
Directories of interest

stress, sleep, exercise, deadlines, lighting, audio, charging

'''
path_db = './dataset' + '/rawaccfeat'

# ema
path_ema = path + '/EMA'
path_ema_response = path_ema + '/response'
path_ema_stress = path_ema_response + '/Stress'
path_ema_sleep = path_ema_response + '/Sleep'
path_ema_exercise = path_ema_response + '/Exercise'


# education
path_edu = path + '/education'
path_edu_deadlines = path_edu + '/deadlines.csv'

# sensing

path_sens = path + '/sensing'
path_sens_activity = path_sens + '/activity'
path_sens_dark = path_sens + '/dark'
path_sens_charge = path_sens + '/phonecharge'
path_sens_audio = path_sens + '/audio'

In [5]:
'''
Id's of interest are the ones that recorded: 
                                            stress, 
                                            exercise, 
                                            deadlines.
'''
regex = r'(u\d{2})'

files = os.listdir(path_ema_stress)
ids_with_stress = [re.search(regex, file).group(0) for file in files]

files = os.listdir(path_ema_exercise)
ids_with_exercise = [re.search(regex, file).group(0) for file in files]

df_deadline = pd.read_csv(path_edu_deadlines)
ids_with_deadlines = list(df_deadline['uid'])

uids = set(ids_with_stress + ids_with_exercise + ids_with_deadlines)

uids = {uid for uid in uids if uid in ids_with_deadlines and 
                               uid in ids_with_exercise  and 
                               uid in ids_with_stress}
# uids = list(uids)
len(uids)


44

In [6]:
'''
EMA definitions
'''
files = os.listdir(path_ema)
df_EMA_def = pd.read_json(path_ema + '/' + 'EMA_definition.json')
df_EMA_def
# df_EMA_def.iloc[7]['questions']

,name,questions
0,Social,"[{'options': '[1]0-4 persons, [2]5-9 persons, ..."
1,Class,"[{'options': '', 'question_id': 'course_id', '..."
2,Comment,"[{'options': '', 'question_id': 'comment', 'qu..."
3,Stress,"[{'options': '[1]A little stressed, [2]Definit..."
4,Sleep,"[{'options': '[1]<3, [2]3.5, [3]4, [4]4.5, [5]..."
5,Activity,"[{'options': '[1]0-10%, [2]11-25%, [3]26-50%, ..."
6,Behavior,[{'options': '(Not at all) 1 2 3 4 5 (Extremel...
7,Exercise,"[{'options': '(Yes) 1 2 (No)', 'question_id': ..."
8,Study Spaces,"[{'options': '', 'question_id': 'place', 'ques..."
9,Do Campbell's jokes suck?,"[{'options': '[1]Great, [2]OK, [3]Terrible, ',..."


In [7]:
'''
Loading stress files
'''
files = os.listdir(path_ema_stress)
df_s = pd.read_json(path_ema_stress + '/' + files[0])
df_s

,level,location,null,resp_time
0,NaN,NaN,"43.75908069,-72.32885314",2013-03-24 08:40:01
1,NaN,NaN,"43.75908069,-72.32885314",2013-03-24 08:40:58
2,NaN,NaN,"43.70677151,-72.28746626",2013-03-25 02:16:45
3,NaN,NaN,1,2013-03-25 00:00:18
4,NaN,NaN,1,2013-03-25 02:16:49
5,2.0,"43.70692415,-72.2873929",NaN,2013-03-25 18:54:56
6,2.0,"43.70555193,-72.28704778",NaN,2013-03-26 03:33:26
7,2.0,"43.70555193,-72.28704778",NaN,2013-03-26 03:33:34
8,1.0,"43.70678675,-72.28732051",NaN,2013-03-27 01:12:20
9,4.0,"43.70032217,-72.28941584",NaN,2013-03-28 22:19:12


In [8]:
'''
Loading exercise files
'''

files = os.listdir(path_ema_exercise)
ids_with_exercise 

df_EMA_exercise_sample = pd.read_json(path_ema_exercise + '/' + files[0])
list(df_EMA_exercise_sample['resp_time'][:].astype(np.int64) // 10 ** 9)


[1364960606,
 1364828138,
 1364836376,
 1364875433,
 1365035283,
 1365041024,
 1365214761,
 1365129591,
 1365194271,
 1365359708,
 1365299772,
 1365571498,
 1366230462,
 1365807244,
 1366166684,
 1366250666,
 1366345779,
 1366604608,
 1366783707,
 1367203093,
 1367796624,
 1368036983,
 1368677250,
 1368767527,
 1368042700,
 1368915663,
 1369001965,
 1370032055,
 1369512401,
 1369772697,
 1369861447,
 1369944425,
 1370117845,
 1370386862,
 1370474328,
 1370228689,
 1375840036]

In [9]:
'''
Loading activity dataset
'''

files = os.listdir(path_sens_activity)

df_s = pd.read_csv(path_sens_activity + '/' + files[0])
df_s

,timestamp,activity inference
0,1364356801,0
1,1364356804,0
2,1364356807,0
3,1364356809,0
4,1364356992,0
5,1364356995,0
6,1364356997,0
7,1364357000,0
8,1364357002,0
9,1364357005,0


In [10]:
'''
Loading light dataset
'''

files = os.listdir(path_sens_dark)

df_s = pd.read_csv(path_sens_dark + '/' + files[0])
df_s


,start,end
0,1364365266,1364385049
1,1364386302,1364391007
2,1364406347,1364411324
3,1364425558,1364429485
4,1364497400,1364502343
5,1364520813,1364529323
6,1364541482,1364556077
7,1364558746,1364563171
8,1364570994,1364574706
9,1364579064,1364584067


In [11]:
# files = os.listdir(path_db)
# conn = sqlite3.connect(path_db + '/' + files[0])
# c =  conn.cursor()
# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())

In [12]:
# c.execute("SELECT * FROM accfeat;")
# row = c.fetchall()
# conn.commit()

In [13]:
'''
Creating Dataframe

uid, timeofday, stress, sleep, time to deadline, #of deadlines, moving, silent,
dark, charging
'''
cols = ['uid', 'time of day', 'stress', 'sleep', 'time to deadline', '# of deadlines',
       'moving', 'silent', 'dark', 'charging']

# data_df = pd.DataFrame([[0] * len(cols)] ,columns=cols)
data_df = pd.DataFrame()
val_missing = -1
for uid in uids:
    # getting stress value
    files = os.listdir(path_ema_stress)
    idx = [i for i in range(len(files)) if uid in files[i]][-1]
    
    df_stress = pd.read_json(path_ema_stress + '/' + files[idx])
    if df_stress.empty:
        continue
    # sleep file
    
    files = os.listdir(path_ema_sleep)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_sleep = pd.read_json(path_ema_sleep + '/' + files[t_idx])
    
    if df_sleep.empty:
        continue
    # exercise file
    
    files = os.listdir(path_ema_exercise)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_exercise = pd.read_json(path_ema_exercise + '/' + files[t_idx])
    
    if df_exercise.empty:
        continue
    # deadline file

    df_deadline = pd.read_csv(path_edu_deadlines)

    deadlines = df_deadline[df_deadline['uid'] == uid]

    t_deadlines = deadlines.columns.values[1:]

    m = lambda d: datetime.datetime.strptime(d, "%Y-%m-%d")

    t_deadlines = list(map(m, t_deadlines))

    # activity files
    files = os.listdir(path_sens_activity)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]

    df_activity = pd.read_csv(path_sens_activity + '/' + files[t_idx])
    
    # audio files
    files = os.listdir(path_sens_audio)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1] 
    df_audio = pd.read_csv(path_sens_audio + '/' + files[t_idx])
    
    # dark
    files = os.listdir(path_sens_dark)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_dark = pd.read_csv(path_sens_dark + '/' + files[t_idx])

    # charging
    
    files = os.listdir(path_sens_charge)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]    
    df_charging = pd.read_csv(path_sens_charge + '/' + files[t_idx])
    
    
    for i in range(len(df_stress)):
        if (math.isnan(df_stress.iloc[i]['level'])):
            continue
        
        t = {'uid': 0, 'time of day': 0, 'stress': 0, 'sleep': 0, 'exercise': 0, 'time to deadline': 0, '# of deadlines': 0,
       'moving': 0, 'silent': 0, 'dark': 0, 'charging': 0}
        
        stress_time = df_stress.iloc[i]['resp_time']
        stress_dt = datetime.datetime.strptime(stress_time.strftime("%Y-%m-%d"), "%Y-%m-%d") #fromtimestamp(stress_time.value // 10 ** 9)

        t['uid'] = uid

        t['stress'] = df_stress.iloc[i]['level']
        
        # getting timeofday
        
        h = stress_time.hour
        t['time of day'] = h // 6 # 0 - 6 : 0, 6 - 12: 1, 12 - 18: 2, 18 - 24: 3
        
        # sleep
        
        t_idx = [i for i in range(len(df_sleep['resp_time'])) if df_sleep['resp_time'].iloc[i].day == stress_time.day and 
                df_sleep['resp_time'].iloc[i].month == stress_time.month]
        if t_idx == []:
            continue
        
        t['sleep'] = df_sleep['hour'][t_idx[-1]]
        
        # Exercise
        vals = df_exercise[(df_exercise['resp_time'].apply(lambda dt: dt.day) == stress_time.day) & 
                           (df_exercise['resp_time'].apply(lambda dt: dt.month) == stress_time.month)]
        if (len(vals) > 0):
            t['exercise'] = vals.iloc[0]['exercise']
        else:
            continue
        
        # time to deadline (taking closest next deadline) 
        # find earliest time to deadline & count number of deadlines starting from response time of stress
        c = 0
        c_min = float("inf")
        min_idx = 0
        for j in range(len(t_deadlines)):
            if (len(deadlines[t_deadlines[j].strftime("%Y-%m-%d")]) == 0 or
                math.isnan(deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0]) or 
                deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0] == 0):
                continue
            if ((t_deadlines[j] - stress_dt).days >= 0 and  (t_deadlines[j] - stress_dt).days < c_min):
                c_min = (t_deadlines[j] - stress_dt).days
                min_idx = j
            if (deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0] > 0):
                c += deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0]
        
        # skip instances where no deadline is available
        if (c_min == float("inf")):
            continue
        
        t['time to deadline'] = c_min
        t['# of deadlines'] = c

        if (math.isnan(c)):
            pdb.set_trace()
        # moving or not

        stress_ut = stress_time.value // 10 ** 9
        
    
        vals = df_activity[df_activity['timestamp'] >= stress_ut]
        if (len(vals) > 0):
            t['moving'] = vals.iloc[0][' activity inference'] != 0
        else:
            t['moving'] = val_missing
        

        vals = df_audio[stress_ut <= df_audio['timestamp']]
        if (len(vals) > 0):
            t['silent'] = vals.iloc[0][' audio inference'] != 0
        else:
            t['silent'] = val_missing

        
        t['dark'] = len(df_dark[(df_dark['start'] <= stress_ut) & (df_dark['end'] >= stress_ut)]) != 0
    

        t['charging'] = len(df_charging[(df_charging['start'] <= stress_ut) & (df_charging['end'] >= stress_ut)]) != 0
        
        data_df = data_df.append(t, ignore_index=True)


In [14]:
 data_df

,# of deadlines,charging,dark,exercise,moving,silent,sleep,stress,time of day,time to deadline,uid
0,52.0,0.0,1.0,4.0,0.0,0.0,7.0,1.0,2.0,3.0,u59
1,52.0,0.0,1.0,1.0,0.0,1.0,5.0,5.0,0.0,1.0,u59
2,52.0,0.0,1.0,1.0,0.0,1.0,5.0,5.0,3.0,2.0,u59
3,52.0,0.0,1.0,1.0,0.0,1.0,5.0,1.0,0.0,2.0,u59
4,52.0,0.0,1.0,1.0,1.0,1.0,5.0,4.0,3.0,2.0,u59
5,52.0,0.0,1.0,1.0,1.0,1.0,5.0,4.0,2.0,2.0,u59
6,52.0,0.0,1.0,1.0,0.0,1.0,5.0,1.0,0.0,1.0,u59
7,52.0,0.0,0.0,4.0,1.0,1.0,7.0,4.0,0.0,3.0,u59
8,52.0,0.0,0.0,4.0,0.0,1.0,7.0,1.0,0.0,3.0,u59
9,52.0,1.0,1.0,4.0,1.0,1.0,7.0,1.0,1.0,3.0,u59


## Random Forest

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [16]:
cols = data_df.columns.values
train_cols = [col for col in cols if (col != 'stress') and (col != 'uid')]

train_df = data_df[:][train_cols]
label_df = data_df[:]['stress']


In [17]:
train_df.head()

,# of deadlines,charging,dark,exercise,moving,silent,sleep,time of day,time to deadline
0,52.0,0.0,1.0,4.0,0.0,0.0,7.0,2.0,3.0
1,52.0,0.0,1.0,1.0,0.0,1.0,5.0,0.0,1.0
2,52.0,0.0,1.0,1.0,0.0,1.0,5.0,3.0,2.0
3,52.0,0.0,1.0,1.0,0.0,1.0,5.0,0.0,2.0
4,52.0,0.0,1.0,1.0,1.0,1.0,5.0,3.0,2.0


In [18]:
label_df.head()

0    1.0
1    5.0
2    5.0
3    1.0
4    4.0
Name: stress, dtype: float64

In [19]:
train_data = train_df.values
label_data = label_df.values

In [20]:
np.where(np.isinf(train_data))

(array([], dtype=int64), array([], dtype=int64))

In [21]:
np.where(np.isinf(label_data))

(array([], dtype=int64),)

In [25]:
train_data.shape[1]

9

In [ ]:
delete_class = 2
delete_idx = np.delete()

In [36]:
# clf = RandomForestClassifier(n_estimators=300, class_weight='auto', criterion='entropy',
#                              max_features=train_data.shape[1], min_samples_split=2)

clf = RandomForestRegressor(n_estimators=300, criterion='mse',
                             max_features=train_data.shape[1], min_samples_split=2)
X_train, X_test, Y_train, Y_test = train_test_split(train_data, label_data, 
                                                    test_size=0.2, random_state=0)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)
# clf.predict(X_test[3].reshape(1, -1))
# clf = clf.fit(X_train, Y_train)
# clf.predict(X_test[3].reshape(1, -1))
# scores = cross_val_score(clf, train_data, label_data, cv=5)
# scores

-0.075190119657249088

In [37]:
clf.predict(X_test)

array([ 2.92825   ,  2.82329365,  2.609     ,  1.41333333,  3.97133333,
        4.4975    ,  1.92666667,  1.96888889,  1.43266667,  2.46772222,
        2.93777778,  1.77388889,  2.17107011,  2.87      ,  1.56628571,
        1.11333333,  2.83388889,  1.75715873,  1.78      ,  2.30844444,
        1.83955556,  2.15633333,  2.86      ,  2.89999206,  2.89999206,
        2.88575   ,  1.21333333,  3.151     ,  1.615     ,  2.47511111,
        2.38979365,  2.80515212,  3.75111111,  1.43416667,  2.55277778,
        2.76157143,  1.17333333,  2.51333333,  2.67      ,  1.85061111,
        2.36111111,  2.7425    ,  4.22633333,  1.08333333,  2.04291667,
        2.25083333,  2.74420635,  2.10933333,  1.94087302,  2.015     ,
        1.97166667,  2.76305556,  2.17107011,  3.20666667,  2.40611111,
        1.59166667,  1.955     ,  1.80192063,  2.11935714,  1.93538889,
        2.91083333,  2.15942857,  2.83466667,  2.12405556,  3.46929365,
        2.14134921,  1.09      ,  2.94666667,  2.22871429,  1.08

In [42]:
Y_test

array([ 1.,  1.,  1.,  1.,  5.,  2.,  3.,  1.,  1.,  5.,  5.,  4.,  4.,
        4.,  3.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  3.,  1.,  1.,  4.,  1.,  1.,  2.,  3.,  5.,  1.,  1.,
        4.,  5.,  1.,  4.,  1.,  3.,  2.,  4.,  1.,  2.,  2.,  1.,  4.,
        1.,  5.,  2.,  2.,  2.,  1.,  3.,  2.,  4.,  2.,  2.,  2.,  4.,
        1.,  1.,  3.,  1.,  1.,  1.,  5.,  4.,  1.,  3.,  4.,  4.,  3.,
        1.,  4.,  2.,  5.,  4.,  5.,  4.,  1.,  1.,  4.,  3.,  1.,  2.,
        4.,  2.,  1.,  1.,  1.,  4.,  1.,  1.,  1.,  1.,  4.,  1.,  2.,
        4.,  1.,  4.,  1.,  1.,  2.,  1.,  2.,  5.,  1.,  2.,  1.,  1.,
        3.,  4.,  1.,  3.,  4.,  5.,  2.,  1.,  1.,  1.,  2.,  1.,  2.,
        4.,  4.,  1.,  1.,  1.,  4.,  1.,  1.,  2.,  3.,  2.,  1.,  1.])

In [41]:
(Y_test - clf.predict(X_test)) // 10

array([-1., -1., -1., -1.,  0., -1.,  0., -1., -1.,  0.,  0.,  0.,  0.,
        0.,  0., -1., -1., -1., -1., -1.,  0., -1., -1., -1., -1., -1.,
       -1., -1.,  0., -1., -1.,  0., -1., -1., -1.,  0.,  0., -1., -1.,
        0.,  0., -1., -1., -1.,  0., -1.,  0., -1.,  0., -1., -1.,  0.,
       -1.,  0., -1.,  0.,  0., -1.,  0.,  0.,  0., -1., -1., -1.,  0.,
       -1., -1.,  0., -1., -1., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.,
       -1.,  0.,  0.,  0.,  0.,  0., -1., -1., -1.,  0.,  0., -1., -1.,
        0.,  0., -1., -1., -1.,  0., -1., -1., -1., -1.,  0., -1.,  0.,
        0., -1.,  0., -0., -1.,  0., -1.,  0.,  0., -1., -1., -1., -1.,
        0.,  0., -1.,  0.,  0.,  0., -1., -1., -1., -1.,  0., -1.,  0.,
        0.,  0., -1., -1., -1.,  0., -1., -1., -1.,  0.,  0., -1., -1.])

In [ ]:
clf.predict(train_data[0].reshape(1, -1))

In [ ]:
label_data[0]